# **FNSPID Experiment**

**Part 1. Imports, Drive Mount, Paths, and Runtime Settings**

In [ ]:
import pandas as pd
import time
import csv
import os
import re
import math
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from google import genai

import math   # Added to avoid missing import
import random

from google.colab import drive
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Google Drive mount failed: {e}")

BASE_PATH = "/content/drive/My Drive/P2/"
NEWS_DATA_PATH = BASE_PATH + "djia_news_cleaned_no_duplicates.csv"
SECTOR_PATH = BASE_PATH + "final_ticker_sector3.csv"
READY_DATA_PATH = BASE_PATH + "djia_news_ready_for_sentiment.csv"
SAMPLE_OUTPUT_PATH = BASE_PATH + "fnspid_sample_multi_prompt_v2.csv"
FULL_OUTPUT_FILE = BASE_PATH + "fnspid_full_sentiment_results.csv"

BATCH_SIZE = 1000   # Number of news rows per batch
MAX_WORKERS = 3     # Number of concurrent threads

api_key = input("Enter your Google GENAI API Key: ").strip()
client = genai.Client(api_key=api_key)

**Part 2. Selected Prompts for FNSPID**

In [ ]:
SELECTED_PROMPTS = {
    "ZS-3": """Act as a sentiment analysis model trained on financial news headlines.
Classify the sentiment of the headline: "{headline}".
Constraint: Answer with exactly one word: Positive, Negative, or Neutral.""",

    "RP-3": """Act as a financial expert. Classify the sentiment for {target} based only on the headline "{headline}".
Constraint: Answer with exactly one word: Positive, Negative, or Neutral.""",

    "CoT-1": """I will offer you a news headline regarding {target}: "{headline}".
Please think step by step:
1. Analyze the rationale and potential impact on the stock price.
2. Identify the sentiment.
Constraint: Return your response in this specific format: Rationale: [Your reasoning] Sentiment: [Positive, Negative, or Neutral]"""
}

**Part 3. Load News Data and Inject Sector Metadata**

In [ ]:
# Data preparation
print("\nReading input data...")
df_news = pd.read_csv(NEWS_DATA_PATH)
df_sector = pd.read_csv(SECTOR_PATH)

# Normalize tickers
df_news['ticker'] = df_news['ticker'].astype(str).str.strip().str.upper()
df_sector['ticker'] = df_sector['ticker'].astype(str).str.strip().str.upper()

# Merge sector information
print("Merging sector information...")
df_merged = df_news.merge(df_sector, on='ticker', how='left')
df_merged['sector'] = df_merged['sector'].fillna('General')

# Manual sector patch for known DJIA tickers
manual_sector_map = {
    "AMGN": "Healthcare",
    "CVX": "Energy",
    "DIS": "Communication Services",
    "DOW": "Materials",
    "IBM": "Technology",
    "MMM": "Industrials",
    "MRK": "Healthcare",
    "TRV": "Financials",
    "WBA": "Consumer Staples"
}

mask = df_merged['ticker'].isin(manual_sector_map)
df_merged.loc[mask, 'sector'] = df_merged.loc[mask, 'ticker'].map(manual_sector_map)

print("\nSector distribution (Top 10):")
print(df_merged['sector'].value_counts().head(10))
print(f"Remaining 'General' count: {len(df_merged[df_merged['sector'] == 'General'])}")

# Save the ready-to-run dataset
df_merged.to_csv(READY_DATA_PATH, index=False)
print("\n" + "="*40)
print(f"Preprocessing completed. Saved to:\n{READY_DATA_PATH}")
print("="*40)

**Part 4. Sample Run Setup (50 News)**

In [ ]:
print("\nSampling 50 news rows for prompt testing...")
df_ready = pd.read_csv(READY_DATA_PATH)

SAMPLE_SIZE = 50
df_sample = df_ready.sample(n=SAMPLE_SIZE, random_state=42).copy()

# Preserve original index as news_id
if 'news_id' not in df_sample.columns:
    df_sample['news_id'] = df_sample.index

print(f"Sample is ready: {len(df_sample)} rows")

**Part 5. Output Parsing: Extract Sentiment Label from Raw Response**

In [ ]:
def clean_sentiment_label(raw_text):
    if not raw_text:
        return "Unknown"

    txt = str(raw_text).lower().strip()

    # Match CoT-style output containing "Sentiment: ..."
    match = re.search(r"sentiment[^a-zA-Z]*?(positive|negative|neutral)", txt)
    if match:
        return match.group(1).capitalize()

    # Fallback: match explicit sentiment words
    if re.search(r"\bpositive\b", txt):
        return "Positive"
    if re.search(r"\bnegative\b", txt):
        return "Negative"
    if re.search(r"\bneutral\b", txt):
        return "Neutral"

    return "Unknown"

**Part 6. Single Task Inference Function (One News × One Prompt)**

In [ ]:
def process_single_task(task):
    row = task['row']
    pid = task['prompt_id']
    template = task['template']

    headline = str(row['title']) if pd.notna(row['title']) else ""
    headline = headline.strip()

    target = str(row['ticker']).strip()
    sector = str(row['sector']).strip()

    # Format the prompt safely
    try:
        prompt_text = template.format(headline=headline, target=target, sector=sector)
    except:
        prompt_text = f"{template}\nHeadline: {headline}"

    # Retry up to 3 times
    for attempt in range(3):
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt_text
            )
            try:
                raw_text = response.candidates[0].content.parts[0].text
            except:
                raw_text = response.text

            parsed_label = clean_sentiment_label(raw_text)

            return {
                "news_id": row['news_id'],
                "date": row['date'],
                "ticker": target,
                "headline": headline,
                "sector": sector,
                "prompt_id": pid,
                "parsed_sentiment": parsed_label,
                "raw_response": raw_text,
                "success": True
            }

        except Exception as e:
            err = str(e).lower()
            if "429" in err or "quota" in err or "exceed" in err:
                time.sleep(2 * (attempt + 1))
            else:
                time.sleep(1)

    return {
        "news_id": row['news_id'],
        "prompt_id": pid,
        "success": False,
        "error": "Failed"
    }

**Part 7. Sample Multi-Prompt Execution and Save Output**

In [ ]:
print(f"\nRunning multi-prompt sample test: each news × {len(SELECTED_PROMPTS)} prompts...")

tasks = []
for _, row in df_sample.iterrows():
    for pid, template in SELECTED_PROMPTS.items():
        tasks.append({"row": row, "prompt_id": pid, "template": template})

results = []
MAX_WORKERS = 3

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(process_single_task, t) for t in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        results.append(future.result())

# Save sample results
df_result = pd.DataFrame(results)
df_result = df_result.sort_values(by=['news_id', 'prompt_id'])
df_result.to_csv(SAMPLE_OUTPUT_PATH, index=False)

print("\n" + "="*40)
print(f"Pipeline completed. Results saved to:\n{SAMPLE_OUTPUT_PATH}")
print("="*40)

print("\nPreview (first 6 rows):")
print(df_result[['news_id', 'ticker', 'prompt_id', 'parsed_sentiment']].head(6))

**Part 8. Full Run (Batch Processing + Failure Handling + Resume Logic)**

In [ ]:
import logging
import datetime

print("\n=== Full Run (Production Run with Failure Handling) ===")

LOG_FILE = BASE_PATH + "run_log.txt"
FAILURE_FILE = BASE_PATH + "fnspid_failures.csv"  # Failure record file

logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True
)
console = logging.StreamHandler()
console.setLevel(logging.WARNING)
logging.getLogger('').addHandler(console)

# Initialize failure record file
if not os.path.exists(FAILURE_FILE):
    with open(FAILURE_FILE, 'w', newline='', encoding='utf-8') as f:
        csv.writer(f).writerow(['news_id', 'ticker', 'prompt_id', 'error_msg', 'timestamp'])

def clean_sentiment_label(raw_text):
    if not raw_text:
        return "Unknown"
    txt = str(raw_text).lower().strip()

    # Match CoT-style output containing "Sentiment: ..."
    match = re.search(r"sentiment[^a-zA-Z]*?(positive|negative|neutral)", txt)
    if match:
        return match.group(1).capitalize()

    # Fallback: match explicit sentiment words
    if re.search(r"\bpositive\b", txt):
        return "Positive"
    if re.search(r"\bnegative\b", txt):
        return "Negative"
    if re.search(r"\bneutral\b", txt):
        return "Neutral"

    return "Unknown"

# Load full dataset
df_ready = pd.read_csv(READY_DATA_PATH)
if 'news_id' not in df_ready.columns:
    df_ready['news_id'] = df_ready.index

total_news = len(df_ready)
num_prompts = len(SELECTED_PROMPTS)
TOTAL_TASKS_GLOBAL = total_news * num_prompts
num_batches = math.ceil(total_news / BATCH_SIZE)

# Initialize main output file
if not os.path.exists(FULL_OUTPUT_FILE):
    header = ['news_id', 'date', 'ticker', 'sector', 'headline', 'prompt_id', 'parsed_sentiment', 'raw_response', 'success']
    with open(FULL_OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
        csv.writer(f).writerow(header)

def process_task_for_full_run(task):
    row = task['row']
    pid = task['prompt_id']
    template = task['template']

    headline = str(row['title']) if pd.notna(row['title']) else ""
    headline = headline.strip()
    target = str(row['ticker']).strip()
    sector = str(row['sector']).strip()

    try:
        prompt = template.format(headline=headline, target=target, sector=sector)
    except:
        prompt = f"{template}\nHeadline: {headline}"

    last_error = ""
    for attempt in range(3):
        try:
            res = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)
            try:
                raw = res.candidates[0].content.parts[0].text
            except:
                raw = res.text

            label = clean_sentiment_label(raw)
            time.sleep(0.05)
            return [row['news_id'], row['date'], target, sector, headline, pid, label, raw, True]

        except Exception as e:
            last_error = str(e)
            err_msg = last_error.lower()
            if "429" in err_msg or "quota" in err_msg or "exceed" in err_msg:
                delay = (2 ** attempt) + random.uniform(0, 1)
                if attempt == 0:
                    logging.warning(f"Rate Limit (ID: {row['news_id']})")
                time.sleep(delay)
            else:
                logging.error(f"API Error (ID: {row['news_id']}): {e}")
                time.sleep(1)

    # On failure, append to failure file
    try:
        with open(FAILURE_FILE, 'a', newline='', encoding='utf-8') as ff:
            csv.writer(ff).writerow([
                row['news_id'], target, pid, last_error, datetime.datetime.now()
            ])
    except:
        pass

    logging.error(f"Failed (ID: {row['news_id']}, {pid}): {last_error}")
    return [row['news_id'], row['date'], target, sector, headline, pid, "Error", f"FAILED: {last_error}", False]

# Batch loop with resume logic (only considers success=True as completed)
for batch_idx in range(num_batches):
    start_idx = batch_idx * BATCH_SIZE
    end_idx = min((batch_idx + 1) * BATCH_SIZE, total_news)

    processed_keys = set()
    completed_count = 0
    if os.path.exists(FULL_OUTPUT_FILE):
        try:
            df_done = pd.read_csv(FULL_OUTPUT_FILE, usecols=['news_id', 'prompt_id', 'success'])
            df_success = df_done[df_done['success'] == True]
            processed_keys = set(df_success['news_id'].astype(str) + "_" + df_success['prompt_id'])
            completed_count = len(processed_keys)
        except:
            pass

    progress_pct = (completed_count / TOTAL_TASKS_GLOBAL) * 100

    print("\n" + "="*60)
    print(f"Batch {batch_idx + 1}/{num_batches} | News rows: {start_idx}-{end_idx - 1}")
    print(f"[Valid progress]: {completed_count}/{TOTAL_TASKS_GLOBAL} ({progress_pct:.2f}%)")
    print("="*60)

    df_batch = df_ready.iloc[start_idx:end_idx]
    tasks = []
    for _, row in df_batch.iterrows():
        for pid, template in SELECTED_PROMPTS.items():
            task_key = f"{row['news_id']}_{pid}"
            if task_key not in processed_keys:
                tasks.append({"row": row, "prompt_id": pid, "template": template})

    if len(tasks) == 0:
        print("[Skip] All tasks in this batch were already completed successfully.")
        continue

    print(f"Pending tasks: {len(tasks)} (including retries for previously failed tasks)")

    with open(FULL_OUTPUT_FILE, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = [executor.submit(process_task_for_full_run, t) for t in tasks]

            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Batch {batch_idx+1}"):
                result = future.result()
                writer.writerow(result)
                f.flush()

print("\nRun completed.")
print(f"Result file: {FULL_OUTPUT_FILE}")
print(f"Failure file: {FAILURE_FILE} (if any)")
print(f"Log file: {LOG_FILE}")

# **FNSPID Sentiment Result**

**Part 1. Setup, File Paths, and Data Loading**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive
from statsmodels.tsa.stattools import grangercausalitytests


# 1. Initialization
BASE_PATH = "/content/drive/MyDrive/P2/"
SENTIMENT_FILE = os.path.join(BASE_PATH, "fnspid_full_sentiment_results.csv")
DJIA_FILE = os.path.join(BASE_PATH, "DJIA_2021_2023.csv")

# Output paths
OUTPUT_BEST_FILE = os.path.join(BASE_PATH, "Final_Dataset_Predictive_Best_1221.csv")
OUTPUT_LEADERBOARD = os.path.join(BASE_PATH, "Final_Leaderboard.csv")

# Load files
df_sent_raw = pd.read_csv(SENTIMENT_FILE)
df_djia_raw = pd.read_csv(DJIA_FILE)

**Part 2. Preprocessing: Dates, Sentiment Scoring, and Returns**

In [ ]:
# Preprocessing: standardize date columns
df_sent_raw['date'] = pd.to_datetime(df_sent_raw['date'], utc=True).dt.date
df_djia_raw['date'] = pd.to_datetime(df_djia_raw['date']).dt.date

# Map sentiment labels to numeric scores
sentiment_map = {'Positive': 1, 'Negative': -1, 'Neutral': 0}
df_sent_raw['score'] = df_sent_raw['parsed_sentiment'].map(sentiment_map).fillna(0)

# Prepare price data and returns
df_price = df_djia_raw.sort_values('date').copy()
df_price['Return'] = df_price['close'].pct_change()
df_price['Next_Return'] = df_price['Return'].shift(-1)      # T+1
df_price['Next_Return_T2'] = df_price['Return'].shift(-2)   # T+2 (backup)

**Part 3. Prompt-Level Evaluation and Leaderboard Construction**

In [ ]:
print("Computing prompt performance and generating leaderboard...")

all_prompts = df_sent_raw['prompt_id'].unique()
table_rows = []  # Store leaderboard rows

for pid in all_prompts:
    # Filter by prompt_id
    sub_df = df_sent_raw[df_sent_raw['prompt_id'] == pid].copy()

    # Aggregate to daily counts and sentiment composition
    daily = sub_df.groupby('date').agg(
        total=('news_id', 'count'),
        pos=('score', lambda x: (x == 1).sum()),
        neg=('score', lambda x: (x == -1).sum())
    ).reset_index()

    # Compute DMSI (Daily Market Sentiment Index)
    daily['DMSI'] = (daily['pos'] - daily['neg']) / daily['total']

    # Merge with price data
    merged = pd.merge(df_price, daily, on='date', how='inner').dropna()

    if len(merged) > 30:
        # Core metric: T+1 IC (correlation between DMSI and next-day return)
        ic_next = merged['DMSI'].corr(merged['Next_Return'])

        # Store intermediate results for deeper analysis later
        row_data = {
            'Prompt': pid,
            'T+1 IC (Predictive)': ic_next,
            'Granger p-value': '-',     # Placeholder (computed only for the best prompt)
            'T+2 IC (Safety)': '-',     # Placeholder (computed only for the best prompt)
            'Description': 'Baselines', # Will be overwritten below if matched
            '_data': merged             # Cache merged data for the best prompt
        }

        # Assign a simple strategy label based on the prompt id
        if 'ZS' in pid:
            row_data['Description'] = 'Zero-Shot'
        if 'CoT' in pid:
            row_data['Description'] = 'Chain-of-Thought'
        if 'RP' in pid:
            row_data['Description'] = 'Role-Playing'

        table_rows.append(row_data)

# Build leaderboard table and sort by absolute predictive IC
df_table = pd.DataFrame(table_rows)
df_table = df_table.sort_values('T+1 IC (Predictive)', key=abs, ascending=False).reset_index(drop=True)

**Part 4. Deep Validation for the Best Prompt: T+2 IC and Granger Test**

In [ ]:
print("\nRunning deeper validation for the top-ranked prompt...")

# Extract best prompt row
best_row_idx = 0
best_data = df_table.loc[best_row_idx, '_data']

# Compute T+2 IC (safety check)
ic_t2 = best_data['DMSI'].corr(best_data['Next_Return_T2'])
df_table.loc[best_row_idx, 'T+2 IC (Safety)'] = f"{ic_t2:.4f}"

# Compute Granger causality p-value (lag=1)
gc_res = grangercausalitytests(best_data[['Return', 'DMSI']], maxlag=1, verbose=False)
p_val = gc_res[1][0]['ssr_ftest'][1]
p_str = "< 0.001" if p_val < 0.001 else f"{p_val:.4f}"
df_table.loc[best_row_idx, 'Granger p-value'] = p_str

# Mark the best prompt for display only
df_table.loc[best_row_idx, 'Prompt'] = f"**{df_table.loc[best_row_idx, 'Prompt']}**"

**Part 5. Save Outputs: Best Dataset and Leaderboard**

In [ ]:
print("\nSaving output files...")

# Save best dataset (used for Streamlit and plotting)
best_data.to_csv(OUTPUT_BEST_FILE, index=False)
print(f"Best prompt dataset saved: {OUTPUT_BEST_FILE}")

# Save leaderboard table (drop cached data column)
final_display_df = df_table.drop(columns=['_data'])
final_display_df.index = final_display_df.index + 1
final_display_df.to_csv(OUTPUT_LEADERBOARD, index=True)
print(f"Leaderboard saved: {OUTPUT_LEADERBOARD}")

**Part 6. Display and Plot the Top Prompts by Predictive IC**

In [ ]:
print("\n" + "="*60)
print("Final Leaderboard")
print("="*60)
display(final_display_df)

# Plot top 5 prompts by T+1 IC
plt.figure(figsize=(8, 5))
plot_df = df_table.head(5)
colors = ['#D62728' if i == 0 else '#A9A9A9' for i in range(len(plot_df))]
plt.bar(plot_df['Prompt'].str.replace('**', ''), plot_df['T+1 IC (Predictive)'], color=colors)
plt.title('Predictive Power (T+1 IC) Comparison')
plt.ylabel('Information Coefficient (IC)')
plt.xlabel('Prompt Strategy')
plt.axhline(0.05, color='gray', linestyle='--', alpha=0.5, label='Significance Level')
plt.legend()
plt.tight_layout()
plt.show()

print("\nAll steps completed. You can download the generated CSV files from the P2 folder.")

**Part 7. Simple Backtest: Strategy vs Benchmark Cumulative Returns**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("\nRunning a simple cumulative return backtest...")

# Define a simple long/short signal:
# Go long when DMSI > 0, go short when DMSI < 0
best_data['Signal'] = np.where(best_data['DMSI'] > 0, 1, -1)

# Strategy return = signal * next-day market return
best_data['Strategy_Return'] = best_data['Signal'] * best_data['Next_Return']

# Cumulative returns (compounded)
best_data['Cumulative_Strategy'] = (1 + best_data['Strategy_Return']).cumprod()
best_data['Cumulative_Benchmark'] = (1 + best_data['Next_Return']).cumprod()

# Plot cumulative returns
plt.figure(figsize=(12, 6))

plt.plot(
    best_data['date'],
    best_data['Cumulative_Strategy'],
    label='ZS-3 Sentiment Strategy',
    color='#D62728',
    linewidth=2
)

plt.plot(
    best_data['date'],
    best_data['Cumulative_Benchmark'],
    label='Buy & Hold DJIA (Benchmark)',
    color='gray',
    linestyle='--',
    alpha=0.6
)

plt.title('Cumulative Returns: Sentiment Strategy vs Benchmark', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Normalized Wealth (Start = 1.0)')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)

# Final return multiple
total_return = best_data['Cumulative_Strategy'].iloc[-1] - 1
print("Backtest summary:")
print(f"   - Strategy total return: {total_return * 100:.2f}%")

plt.show()